In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

edited SImple LSTM With Word2Vec link: https://www.kaggle.com/khkuggle/simple-lstm-with-word2vec/notebook 

For Koreans

이번 대회에서 우리는 코멘트들의 유해 정도의 순위를 매기게 될겁니다. 우리는 주어진 코멘트들을 상대적 유해성을 참고하여 점수를 줄겁니다. 높은 유해도을 가진 코멘트는 낮은 유해도를 가진 코멘트보다 높은 점수를 받을 겁니다.

## 라이브러리와 데이터 가져오기 

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import gensim

from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.probability import  FreqDist
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Flatten, Dropout, Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
dff1 = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
dff2 = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
print(dff1.head(10))
print(dff2.head())

## EDA

In [ ]:
print('dff.shape : ',dff1.shape,'\n')
print('dff.isna().sum() : ',dff1.isna().sum(),'\n')
print('dff.describe() :', dff1.describe(),'\n')
print('dff.severe_toxic.value_counts():', dff1.severe_toxic.value_counts(),'\n')

train dataset의 severe_toxic column의 value가 0에 치우쳐져 있습니다.

In [ ]:
dff1['toxicity'] = (dff1[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0).astype(int)
dff1 = dff1[['comment_text', 'toxicity']].rename(columns={'comment_text': 'text'})
dff1.sample(5)

In [ ]:
print('dff.shape : ',dff1.shape,'\n')
print('dff.describe() :', dff1.describe(),'\n')
print('dff.severe_toxic.value_counts():', dff1.toxicity.value_counts(),'\n')

In [ ]:
min_len = (dff1['toxicity'] == 1).sum()
df_undersample = dff1[dff1['toxicity'] == 0].sample(n=min_len, random_state=201)
dff1 = pd.concat([df_undersample, dff1[dff1['toxicity'] == 1]])
dff1 = shuffle(dff1)
dff1.toxicity.value_counts()

In [ ]:
dff1.text = dff1.text.map(lambda x:x.replace('\n',' '))
dff1.text[:2]

In [ ]:
toxic = dff1[dff1['toxicity'] == 1]
not_toxic = dff1[dff1['toxicity'] == 0]

In [ ]:
toxic

In [ ]:
dff2_processed = dff2.drop(['less_toxic'], axis=1)
toxic_text = pd.concat([dff2['more_toxic'], toxic['text']])
toxic_text

In [ ]:
wordcloud = WordCloud(width=1400, height=700, background_color='white').generate(' '.join(toxic_text.tolist()))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud)
plt.axis('off')
plt.title('The most 100 frequent words in the toxic comments', fontsize=50)
plt.tight_layout(pad=0)
plt.savefig('wordcloud_toxic_text.png')
plt.show()

In [ ]:
wordcloud = WordCloud(width=1400, height=700, background_color='white').generate(' '.join(not_toxic.text.tolist()))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud)
plt.axis('off')
plt.title('The most 100 frequent words in the normal comments', fontsize=50)
plt.tight_layout(pad=0)
plt.show()

# 텍스트 전처리

X 그리고 y 변수를 갖고있으니까 이용하기 위해서 복사를 해보자

In [ ]:
dff1.head()

In [ ]:
dff2_text = pd.concat([dff2.less_toxic, dff2.more_toxic])
dff2_toxicity = pd.DataFrame(dff2_text,  columns=['text'])
dff2_toxicity['toxicity'] = 1
dff2_toxicity

In [ ]:
y = pd.concat([dff1.toxicity, dff2_toxicity.toxicity])
dff1_x = dff1.drop('toxicity', axis=1)
dff2_toxicity_x = dff2_toxicity.drop('toxicity', axis=1)
x = pd.concat([dff1_x, dff2_toxicity_x])

In [ ]:
texts = x.copy()
texts.reset_index(inplace = True, drop = True)
texts.head()

In [ ]:
sys.setrecursionlimit(6000)

RecursionError를 방지하기 위해 recursionlimit을 6000으로 늘립니다. RecursionError는 재귀와 관련된 에러입니다. 가장 많이 발생하는 이유는 Python이 정한 최대 재귀 깊이보다 재귀의 깊이가 더 깊어질 때입니다.

우리는 NLP의 필수 과정인 전처리를 할겁니다. 

이번 전처리에서는 정제 작업 그리고 어간 추출을 할겁니다.

어간(Stem)을 추출하는 작업을 어간 추출(stemming)이라고 합니다. 어간 추출은 형태학적 분석을 단순화한 버전이라고 볼 수도 있고, 정해진 규칙만 보고 단어의 어미를 자르는 어림짐작의 작업이라고 볼 수도 있습니다. 다시 말해, 이 작업은 섬세한 작업이 아니기 때문에 어간 추출 후에 나오는 결과 단어는 사전에 존재하지 않는 단어일 수도 있습니다.

예를 들면 이런 식으로 단어가 전처리됩니다.
- formalize → formal
- allowance → allow
- electricical → electric

In [ ]:
ps = PorterStemmer()
corpus = []
for i in tqdm(range(0, len(texts))):
    cleaned = re.sub('[^a-zA-Z]', ' ', texts['text'][i])
    cleaned = cleaned.lower().split()
    
    cleaned = [ps.stem(word) for word in cleaned if not word in stopwords.words('english')]
    cleaned = ' '.join(cleaned)
    corpus.append(cleaned)

In [ ]:
DIM = 100

X = [d.split() for d in corpus]
w2v_model = gensim.models.Word2Vec(sentences = X, vector_size = DIM, window = 10, min_count = 1)

우리가 얼마나 많은 단어들을 처리했는지 확인합니다.

In [ ]:
len(w2v_model.wv.key_to_index.keys())

In [ ]:
w2v_model.wv.most_similar('fuck')

문장들을 토크나이즈하고 말뭉치 X를 순차적인 숫자 데이터로 변환합니다.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X[:3]

모든 문장들의 길이가 20이 되도록 변환해줍니다.

In [ ]:
X = pad_sequences(X, padding = 'pre', maxlen = 20)
X[:10]

우리는 이 벡터들을 모델의 시작하는 weights로 넣고, 더 나은 정확도를 얻기 위해 이 벡터들을 다시 만듭니다.

In [ ]:
vocab_size = len(tokenizer.word_index) + 1 
vocab = tokenizer.word_index

In [ ]:
def get_weights_matrix(model):
    weights_matrix = np.zeros((vocab_size, DIM))
    
    for word, i in vocab.items():
        weights_matrix[i] = model.wv[word]
        
    return weights_matrix

embedding_vectors = get_weights_matrix(w2v_model)

# 모델링 & 학습

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, output_dim = DIM, weights = [embedding_vectors], input_length = 20))
model.add(Dropout(0.2))

model.add(LSTM(64))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = 'accuracy')
model.summary()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

es = EarlyStopping(patience=3, 
                   monitor='loss', 
                   restore_best_weights=True, 
                   mode='min', 
                   verbose=1)

# train the model 
hist = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100,
                 callbacks=es, batch_size = 32)

In [ ]:
plt.style.use('fivethirtyeight')

# visualize the models accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

In [ ]:
sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv") 
new_text = tokenizer.texts_to_sequences(sub.text)
new_text = pad_sequences(new_text, maxlen = 20)
sub['score'] = model.predict(new_text) * 1000 
sub.head()

In [ ]:
sub[['comment_id', 'score']].to_csv("submission.csv", index=False)